In [8]:
import pandas as pd 
import numpy as np
import lightgbm as lgb


In [9]:
df_train = pd.read_pickle("../data/processed/df_train.pickle")

In [10]:
df_test = pd.read_pickle("../data/processed/df_test.pickle")

In [11]:
df_train.shape

(2101060, 15)

In [12]:
df_test.shape

(429803, 13)

In [13]:
def test_train_split_df(df_r, test_percentage=0.25):
    import numpy as np
    df = df_r.copy()
    
    msk = np.random.rand(len(df)) < (1-test_percentage)
    
    train = df[msk]
    test = df[~msk]
    
    
    return train, test
    

In [14]:
def build_features(df):
    
    features_to_extract = [
        "sid", "distance_plan", "eta", "price", "distance_query"
    ]
    
    
    if "click_mode" in df.columns:
        features_to_extract.append("click_mode")
        
    df_X = df[features_to_extract]
    return df_X

In [15]:
df_X = build_features(df_train)

In [16]:
df_test = build_features(df_test)

In [17]:
train, val = test_train_split_df(df_X)

In [28]:
x_train = train.drop(["sid", "click_mode"], axis=1).values
y_train = train.loc[:,"click_mode"].values

x_val = val.drop(["sid", "click_mode"], axis=1).values
y_val = val.loc[:,"click_mode"].values

x_test = df_test.drop(["sid"], axis=1).values

In [103]:
print(x_train.shape)
print(y_train.shape)
print(x_val.shape)
print(y_val.shape)

(1575624, 4)
(1575624,)
(525436, 4)
(525436,)


In [87]:
def lgb_f1_score(y_true, y_pred):
    from sklearn.metrics import f1_score
    preds = y_pred.reshape(len(np.unique(y_true)), -1)
    preds = preds.argmax(axis = 0)
    #print(preds.shape)
    #print(y_true.shape)
    return 'f1', f1_score(y_true, preds,average='weighted'), True

In [73]:

lgb_model = lgb.LGBMClassifier(boosting_type="gbdt",
                               num_leaves=61, reg_alpha=0,
                               reg_lambda=0.01,max_depth=-1,
                               n_estimators=2000, objective='multiclass',
                               subsample=0.8, colsample_bytree=0.8, 
                               subsample_freq=1,min_child_samples = 50,  
                               learning_rate=0.05, random_state=2019, metric="None",n_jobs=-1)


eval_set = [(x_val, y_val)]


lgb_model.fit(x_train, y_train, 
              eval_set=eval_set, 
              eval_metric=lgb_f1_score, 
              verbose=10, 
              early_stopping_rounds=100)



/home/sandro/anaconda3/envs/lbm_mc_bl/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Training until validation scores don't improve for 100 rounds.
[10]	valid_0's f1: 0.357725
[20]	valid_0's f1: 0.359514
[30]	valid_0's f1: 0.360389
[40]	valid_0's f1: 0.361561
[50]	valid_0's f1: 0.36237
[60]	valid_0's f1: 0.36301
[70]	valid_0's f1: 0.363822
[80]	valid_0's f1: 0.364456
[90]	valid_0's f1: 0.365112
[100]	valid_0's f1: 0.365577
[110]	valid_0's f1: 0.366026
[120]	valid_0's f1: 0.366479
[130]	valid_0's f1: 0.366927
[140]	valid_0's f1: 0.367217
[150]	valid_0's f1: 0.367518
[160]	valid_0's f1: 0.367714
[170]	valid_0's f1: 0.367942
[180]	valid_0's f1: 0.368101
[190]	valid_0's f1: 0.368187
[200]	valid_0's f1: 0.368428
[210]	valid_0's f1: 0.368579
[220]	valid_0's f1: 0.36873
[230]	valid_0's f1: 0.3688
[240]	valid_0's f1: 0.369031
[250]	valid_0's f1: 0.369221
[260]	valid_0's f1: 0.369361
[270]	valid_0's f1: 0.369535
[280]	valid_0's f1: 0.36976
[290]	valid_0's f1: 0.369863
[300]	valid_0's f1: 0.370021
[310]	valid_0's f1: 0.370132
[320]	valid_0's f1: 0.370222
[330]	valid_0's f1: 0.37

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=0.8,
        importance_type='split', learning_rate=0.05, max_depth=-1,
        metric='None', min_child_samples=50, min_child_weight=0.001,
        min_split_gain=0.0, n_estimators=2000, n_jobs=-1, num_leaves=61,
        objective='multiclass', random_state=2019, reg_alpha=0,
        reg_lambda=0.01, silent=True, subsample=0.8,
        subsample_for_bin=200000, subsample_freq=1)

In [75]:
import pickle
import os
filename = os.path.join("..", "models", "lgbmodel_2k.pickle")
pickle.dump(lgb_model, open(filename, 'wb'))

In [100]:
df_test.head()

,sid,distance_plan,eta,price,distance_query
0,1112456,465,418,700.0,0.000000
1,1112456,465,140,700.0,0.000000
2,1112456,462,178,700.0,0.000000
3,1413458,28846,4779,600.0,15.638394
4,1413458,18743,3098,700.0,15.638394


In [27]:
x_test = test.drop(["sid", "click_mode"], axis=1).values
y_test = test.loc[:,"click_mode"].values

NameError: name 'test' is not defined

In [3]:
import os
import pickle

In [4]:
filename = os.path.join("..", "models", "lgbmodel_2k.pickle")

loaded_model = pickle.load(open(filename, 'rb'))

In [29]:
y_preds = loaded_model.predict(x_test)

In [30]:
print("HI")

HI


In [32]:
df_test.head()

,sid,distance_plan,eta,price,distance_query
0,1112456,465,418,700.0,0.000000
1,1112456,465,140,700.0,0.000000
2,1112456,462,178,700.0,0.000000
3,1413458,28846,4779,600.0,15.638394
4,1413458,18743,3098,700.0,15.638394


In [39]:
df_test.sid.values[:10]

array([1112456, 1112456, 1112456, 1413458, 1413458, 1413458, 1413458,
       1413458, 1243160, 1243160])

In [36]:
y_preds

(429803,)

In [47]:
a=pd.Series(df_test.sid.values)

In [48]:
b=pd.Series(y_preds)

In [50]:
submission = pd.DataFrame(data={'sid':a, 'yhat':b})

In [55]:
import calendar;
import time;
ts = calendar.timegm(time.gmtime())


1557571666


In [71]:
import calendar;
import time;
ts = calendar.timegm(time.gmtime())

submission.to_csv("../submissions/sub_" + str(ts) + ".csv", index=None, header=None, )

In [72]:
pd.read_csv("../sub")

sub_1557571947.csv
